In [23]:
import sqlite3

import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import math
import time

#!pip install deepface
from deepface import DeepFace
from deepface.commons import functions

In [24]:
model = DeepFace.build_model("Facenet")

In [25]:
facial_img_paths = []
#Available at: https://github.com/serengil/deepface/tree/master/tests/dataset
for root, directory, files in os.walk("../dataset"):
    for file in files:
        if '.jpg' in file:
            facial_img_paths.append(root+"/"+file)

In [27]:
instances = []

for i in tqdm(range(0, len(facial_img_paths))):
    facial_img_path = facial_img_paths[i]
    
    #detect and align
    facial_img = functions.preprocess_face(facial_img_path, target_size = (160, 160))
    
    #represent
    embedding = model.find_embeddings(facial_img)[0]
    
    #store
    instance = []
    instance.append(facial_img_path)
    instance.append(embedding)
    instances.append(instance)

100%|██████████| 63/63 [00:59<00:00,  1.05it/s]


In [28]:
df = pd.DataFrame(instances, columns = ["img_name", "embedding"])

In [29]:
df.head()

,img_name,embedding
0,../dataset/couple.jpg,-1.003564
1,../dataset/img1.jpg,1.057407
2,../dataset/img10.jpg,-0.322842
3,../dataset/img11.jpg,0.285914
4,../dataset/img11_reflection.jpg,0.372023


In [30]:
conn = sqlite3.connect('facialdb.db')

In [45]:
cursor = conn.cursor()

In [46]:
cursor.execute('''drop table if exists face_meta ''')
cursor.execute('''drop table if exists face_embeddings''')

cursor.execute('''create table face_meta (ID INT primary key, IMG_NAME VARCHAR(10), EMBEDDING BLOB)''')
cursor.execute('''create table face_embeddings (FACE_ID INT, DIMENSION INT, VALUE DECIMAL(5, 30))''')

In [47]:
cursor

In [64]:
for index, instance in tqdm(df.iterrows(), total=df.shape[0]):
    img_name = instance["img_name"]
    embeddings = instance["embedding"]
    print(embeddings) 
    
    insert_statement = "INSERT INTO face_meta (ID, IMG_NAME, EMBEDDING) VALUES (?, ?, ?)"
    insert_args = (index, img_name, embeddings)
    cursor.execute(insert_statement, insert_args)
    
    #for i, embedding in enumerate(embeddings):
        #insert_statement = "INSERT INTO face_embeddings (FACE_ID, DIMENSION, VALUE) VALUES (?, ?, ?)"
        #insert_args = (index, i, str(embedding))
        #cursor.execute(insert_statement, insert_args)

  0%|          | 0/63 [00:00<?, ?it/s]

-1.0035638809204102


IntegrityError: UNIQUE constraint failed: face_meta.ID

In [54]:
target_img_path = "../target.png"

In [55]:
target_img_path

'../target.png'

In [56]:
target_img = functions.preprocess_face(target_img_path, target_size = (160, 160))

ValueError: ('Confirm that ', '../target.png', ' exists')